## Properly installing aceflow

Make sure to run the following in the specified order in order to avoid dependency issues. If you already have an environment with pyace and tensorflow running, skip these steps. As always, make a new conda environment with python >=3.10 before running the below commands directly in the terminal or using the cell block below.

In [ ]:
!pip install tensorflow==2.8.0 # install tensorflow
!pip install git+https://github.com/BryantLi-BLI/atomate2.git@feature/mpmorph # install atomate2, will be updated to official atomate2 once the PR is merged
!pip install pymatgen==2024.3.1 # install pymatgen, specific version is required for atomate2 branch feature/mpmorph
!pip install mp-api
!pip install fireworks # install fireworks if you want to use the workflow in fireworks

!git clone https://github.com/ICAMS/TensorPotential.git 
!cd TensorPotential
!pip install --upgrade .
!cd ..
!git clone https://github.com/ICAMS/python-ace.git 
!cd python-ace
!pip install --upgrade .
!cd ..
!pip install git+https://gitlab.com/ase/ase.git@master # install ase from gitlab, the version in pypi is not compatible with the current version of TensorPotential
!pip install protobuf==3.20
!pip install numpy==1.26.4 # install numpy, specific version is required for TensorPotential AND pymatgen to work together for now
# !pip install torch   #uncomment this line if you run into torch error
# !pip install ruamel.yaml==0.17.40 #uncomment this line if you run into a ruamel.yaml error
!git clone https://github.com/vir-k01/aceflow.git #if you haven't already done so
!cd aceflow
!pip install --upgrade .

If you wish to use of GPUs for training the potential (which is significantly faster), make sure you have CUDA corrently configured. For CUDA 12.3, the following code cell should be executed. Check the official TensorFlow docs to find what versions match to the CUDA version configured on your GPU.

In [ ]:
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1

If you plan on using a HPC to run these workflows, check if cudatoolkit and cudnn are already installed, and if so just module load the correct versions. To test if the GPUs are correctly configured, run the below cell:

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU')) # check if the GPU is available

If the result isn't empty, you're all set to train ACE with GPU :D
Do keep in mind that only a single GPU can be used at present, and the memory being used should be monitored by running !nvidia-smi in the terminal.

## Usage

There are 4 main use-cases for aceflow:

- Case 1. To automate the generation of training data suitable for training ACE potentials
- Case 2. To train ACE without much manual intervention, and be able to track training experiments in a systematic manner
- Case 3. To improve the quality of already trained potentials by fine-tuning on additional data, or with active learning
- Case 4. To have an end-to-end worflow that takes in the chemical system of interest and additional data relevant to the problem being studied, and outputs the trained ACE potential. 

This last one is in essense just a combination of 1,2 and 3.

In this case, we assume the potential is first trained for a specific chemical system, on data from "bulk" materials, and is then refined with active learning to capture "unphysical" or far from equilibrium typically configurations not seen in 'bulk" data. The trained potential can then be tuned further for a specific task by supplying data relevant to the application, such as predicting defect formation energies, or interfacial energies, searching for transition states, or even studying nucleation and growth in crystals with long-time MD. 

For now let's consider the example of training a "versatile" potential for the Si-O system. Such an ACE potential has been already been trained in the past, but for the sake of demonstration, let's see how aceflow can be used to train something similar in an automated manner. 

Begin by importing the required functions from aceflow:

In [ ]:
from aceflow.flows.trainer import ACEMaker, ProductionACEMaker
from aceflow.flows.data import DataGenFlowMaker, ActiveStructuresFlowMaker
from aceflow.utils.config import TrainConfig, DataGenConfig, ActiveLearningConfig

### Use Case 1: Automated data generation

For Case 1, we can use the DataGenFlowMaker. Let's say we have a series of compositions of interest in our system. For Si-O, these probably are Si, O2, SiO, SiO2, SiO4. In addition, let's say we have some precomputed data too. Due to the size limitations of accessing stores through fireworks, the precomputed data must first be dumped to a pckl.gzip file (as is customary when training ACE) and the path to this file must be used.

In [ ]:
compositions = ['Si', 'O2', 'SiO', 'SiO2', 'SiO4']
precomputed_data_file = 'data.pckl.gzip' # precomputed data located here, make sure it's either a pandas dataframe or a dict with atleast the keys energy, forces, ase_atoms.

data_config_args = {} 
maker = DataGenFlowMaker(DataGenConfig(**data_config_args))
data_flow = maker.make(compositions=compositions, precomputed_data=precomputed_data_file)

The above flow will generate training data by sampling points about high temperature MD trajectories of the compositions passed to the .make() call, starting from both crystalline structures taken from the Materials Project and amorphous configurations generated using packmol. By default, this returns the energies directly taken from the MD trajectory, which are computed using the default MD set (which in turn sets the INCAR tags) in atomate2. 

Due to computational costs, this set is usually performed with a reduced accuracy (PBESol POTCARS, non-spin polarized, gamma-point only calculation among other things). To access better levels of theory, the 'data_generator' tag can be set to 'Static', as shown above, which then makes the flow also run static calculations on the frames selected from the MD trajectory. The defualt Static set from atomate2 is used here.

In [ ]:
data_config_args = {'data_generator': 'Static'}

In addition, the flow can also generate data by randomly removing atomic pairs (to maintain electroneutrality) from structures generated above,and running Statics to computed the energies of these "defected" configurations too. To do this, set 'data_generator' to 'Static_Defect'. A full description of the tags in DataGenConfig are present in the aceflow.utils.config file

In [ ]:
data_config_args = {'data_generator': 'Static_Defect'}

To have more control over the static calculation, whether to use meta-GGA, tighter cutoffs, etc., just instantiate custom MDMaker and StaticMaker objects with the required set from atomate2 and pass them in the .make() call too:

In [ ]:
from atomate2.vasp.jobs.core import StaticMaker
from atomate2.vasp.jobs.md import MDMaker

md_maker = MDMaker() 
static_maker = StaticMaker() #customize the maker if needed

data_config_args.update({'static_maker': static_maker, 'md_maker': md_maker})

maker = DataGenFlowMaker(DataGenConfig(**data_config_args))
data_flow = maker.make(compositions=compositions, precomputed_data=precomputed_data_file)

As a reminder, the flow can be executed locally, or using a job manager such as fireworks:

In [ ]:
from jobflow import run_locally
from jobflow.managers.fireworks import flow_to_workflow
from fireworks import LaunchPad

local_output = run_locally(data_flow)  # run the data flow locally
#OR
lp = LaunchPad.auto_load()  # load the launchpad
wf = flow_to_workflow(data_flow)  # convert the data flow to a workflow
lp.add_wf(wf)  # add the workflow to the launchpad, and launch it externally using qlaunch, say from an HPC

The data generated can be accessed from the TaskDoc as:

In [ ]:
data = local_output.output.acedata  # get the data from the data flow

### Use Case 2: Systematic ACE training 

When training general ML models, let alone interatomic potentials, a very common problem seen is the almost trial-and-error based approach to optimize hyper-parameters. Although there exist a great many tools to do this efficiently, a very real problem that practitioners (including myself) face is organizing and keeping track of what worked and what didn't. Moreover, when it comes to training ACE potentials, there are some good practices to follow to improve the quality of the model trained, which might take users time to get used to. 

Using jobflow allows to keep track of model training, while also automatically chaining several steps that are routinely followed when training ACE. Once we've generated the data we need using the previous cells, we can get to training the actual model:

In [ ]:
trainer_args = {}
train_maker = ACEMaker(TrainConfig(**trainer_args))
train_flow = train_maker.make(data_flow.output.acedata)

Once again, check the attributes for the TrainConfig object to see the hyperparameters that can be changed when training. The default values work for training potentials for most "bulk" systems. 

A good practice when training ACE is to do the training in multiple steps, first giving a high weight to forces in the loss function, and then giving a more balanced weight to energies and forces. This can be specificed through the trainer_args:

In [ ]:
trainer_args = {'loss_weight': [0.99, 0.3]}

This way, first a high weight (0.99) is given to the forces, and then a second step of training is done with a weight of 0.3 on the forces.
In principle, this can be done for how many ever steps are needed, so [0.99, 0.9, 0.5, 0.3] would complete the training in 4 steps, each of atmost 2000 iterations of training. 

Another use for this is when training a full 2000 steps is not possible due to timing issues on HPCs: for this training can split into more smaller steps by specifying for example: 

In [ ]:
trainer_args = {'loss_weight': [0.99, 0.99, 0.3, 0.3], 'train_steps': 1000}

Once the flow completes executing, the trained potential and ace active_set can be accessed from the TaskDoc:

In [ ]:
potential = train_flow.output.trained_potential # the trained ACE potential
active_set_file = train_flow.output.active_set_file # path to the active set file
potential.dump('potential.yaml') # save the potential to a yaml file for use with ASE/LAMMPS

The flow also has a built-in contingency to restart training if a training step did not complete in the time allocated to it. This can used as:

In [ ]:
trainer_args = {'restart_failed_runs': True}

Finally, heiracical (or ladder) fitting of the potential can be turned on by specifying the type ('body_order' or 'power_order') and the fraction by which the potential is allowed to grow each iteration:

In [ ]:
trainer_args = {'ladder_fitting': 'power_order', 'ladder_step': 0.2} # ladder fitting, size of potential grown by a factor 1.2 at each ladder step

### Use Case 3: Active Learning

A common practice in training good ML potentials is using active learning to improve the model's performance on configurations it does badly on. In the case of ACE, this can be accessed by the extrapolation grade. In aceflow, active learning several active learning strategies have been implemented, and they can be called by instantiating the appropriate class. By default, a random packed sampling approach is used by sampling compositions in the space defined by the compositions provided, and then randomly generating structures of these sampled compositions with packmol. In addition, extremely high temeprature ACE-MD is performed for the structures generated and the extrapolation grade is computed for each structure.

DFT statics are run on all structures with an extrapolation grade > 'gamma_min', < 'gamma_max' (defaults to 5 and 100 respectively). Good practice is to make sure the Static set used here is the same that was used for data generation (in Use Case 1) to ensure compatibility of data.  

In [ ]:
active_learning_args = {}
active_learning_maker = ActiveStructuresFlowMaker(ActiveLearningConfig(**active_learning_args))
active_learning_flow = active_learning_maker.make(data_flow.output.acedata, potential)

The output of this flow is a (hopefully) better potential. This flow can be repeated several times by changing the 'active_leaning_loops' arg, which defaults to 1:

In [ ]:
active_learning_args = {'active_learning_loops': 3, 'gamma_min': 1} # run 3 active learning loops, with a tigher gamma_max threshold

Another poular sampling strategy is to run high temperature MD for the structures of interest. To do this:

In [ ]:
from aceflow.active_learning.core import HighTempMDSampler

sampler = HighTempMDSampler() # customize the sampler if needed
active_learning_args.update({'sampler': sampler})

### Use Case 4: Automated End-to-End training

Finally, for the brave souls out there, all of the above can be automated by instantiating a ProductionACEMaker flow:

In [ ]:
compositions = ['Si', 'O2', 'SiO', 'SiO2', 'SiO4']
precomputed_data = ... # load precomputed data here, make sure it's a dictionary with atleast the keys energy, forces, ase_atoms.

data_config_args = {'data_generator': 'Static'}
trainer_args = {'loss_weight': [0.99, 0.3]}
active_learning_args = {'active_learning_loops': 3, 'gamma_max': 1}

data_config = DataGenConfig(**data_config_args)
train_config = TrainConfig(**trainer_args)
active_learning_config = ActiveLearningConfig(**active_learning_args)

maker = ProductionACEMaker(data_config, train_config, active_learning_config)
flow = maker.make(compositions=compositions)

### Misc: starting from an existing potential or fine-tuning:

Load the potential and simply specify it in the .make() call of either the ProductionACEMaker or ACEMaker flows:

In [ ]:
from aceflow.core.model import TrainedPotential
pretrained_potential_path = 'potential.yaml' # path to a pretrained potential
pretrained_potential = TrainedPotential.load(pretrained_potential_path)

flow = maker.make(compositions=compositions, pretrained_potential=pretrained_potential)